# Import Libraries and Data

In [1]:
#Import Necessary Libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import os
import folium
import json

In [20]:
#Import Data
df = pd.read_csv('/Users/buketoztekin/Documents/Cleaned_Salaries_Data.csv')

In [21]:
# Propt matplotlib visuals to appear in the notebook 

%matplotlib inline

In [22]:
df.head()

,work_year,experience_level,employment_type,job_title,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,2024,MI,FT,Analytics Engineer,59615,CA,100,CA,S
1,2024,SE,FT,AI Developer,252000,US,0,US,M
2,2024,SE,FT,AI Developer,168000,US,0,US,M
3,2024,MI,FT,Data Scientist,199500,US,100,US,M
4,2024,MI,FT,Data Scientist,86700,US,100,US,M


In [28]:
# File path to your JSON file for the world countries
country_geo = r'/Users/buketoztekin/Documents/world-countries.json'

# That's just in case you want to look at the JSON file contents here too:
# Open the JSON file and load it into a dictionary
with open(country_geo, 'r') as f:
    data = json.load(f)

# Print the top-level keys to understand the structure
print(data.keys())

# If the 'features' key exists (common in GeoJSON), let's print the first feature
if 'features' in data:
    print(json.dumps(data['features'][0], indent=2))  # Print the first feature with pretty formatting
else:
    print("The 'features' key does not exist in the GeoJSON file.")


FileNotFoundError: [Errno 2] No such file or directory: '/Users/buketoztekin/Documents/world-countries.json'

In [25]:
# Print the top-level keys to understand the structure
print(data.keys())

# If the 'features' key exists, let's print the first feature
if 'features' in data:
    # Print the first feature to inspect its structure
    print(json.dumps(data['features'][0], indent=2))
else:
    print("The 'features' key does not exist in the GeoJSON file.")

dict_keys(['type', 'features'])


AttributeError: 'DataFrame' object has no attribute 'dumps'

### In the JSON file, the properties are shown as full country names (name : Afghanistan). Therefore, I need to create new columns with country names to match with their ISO codes.

In [6]:
df

,type,features
0,FeatureCollection,"{'type': 'Feature', 'properties': {'name': 'Af..."
1,FeatureCollection,"{'type': 'Feature', 'properties': {'name': 'An..."
2,FeatureCollection,"{'type': 'Feature', 'properties': {'name': 'Al..."
3,FeatureCollection,"{'type': 'Feature', 'properties': {'name': 'Un..."
4,FeatureCollection,"{'type': 'Feature', 'properties': {'name': 'Ar..."
...,...,...
172,FeatureCollection,"{'type': 'Feature', 'properties': {'name': 'We..."
173,FeatureCollection,"{'type': 'Feature', 'properties': {'name': 'Ye..."
174,FeatureCollection,"{'type': 'Feature', 'properties': {'name': 'So..."
175,FeatureCollection,"{'type': 'Feature', 'properties': {'name': 'Za..."


### In order to match the country codes with full names, I will use pycountry library

In [9]:
import pycountry

# Function to convert country code to country name
def get_country_name_from_code(code):
    try:
        country = pycountry.countries.get(alpha_2=code)
        return country.name
    except:
        return code

# Apply to dataframe creating new columns
df['employee_country_name'] = df['employee_residence'].apply(get_country_name_from_code)
df['company_country_name'] = df['company_location'].apply(get_country_name_from_code)

KeyError: 'employee_residence'

In [10]:
df

,type,features
0,FeatureCollection,"{'type': 'Feature', 'properties': {'name': 'Af..."
1,FeatureCollection,"{'type': 'Feature', 'properties': {'name': 'An..."
2,FeatureCollection,"{'type': 'Feature', 'properties': {'name': 'Al..."
3,FeatureCollection,"{'type': 'Feature', 'properties': {'name': 'Un..."
4,FeatureCollection,"{'type': 'Feature', 'properties': {'name': 'Ar..."
...,...,...
172,FeatureCollection,"{'type': 'Feature', 'properties': {'name': 'We..."
173,FeatureCollection,"{'type': 'Feature', 'properties': {'name': 'Ye..."
174,FeatureCollection,"{'type': 'Feature', 'properties': {'name': 'So..."
175,FeatureCollection,"{'type': 'Feature', 'properties': {'name': 'Za..."


## Check for missing values in the new columns

In [11]:
# Check for missing values in 'employee_country_name' and 'company_country_name' columns
missing_values = df[['employee_country_name', 'company_country_name']].isnull().sum()

# Display the result
print(missing_values)

KeyError: "None of [Index(['employee_country_name', 'company_country_name'], dtype='object')] are in the [columns]"

## Convert data types of the new columns

In [ ]:
# Convert 'employee_country_name' and 'company_country_name' to string (object) type
df['employee_country_name'] = df['employee_country_name'].astype(str)
df['company_country_name'] = df['company_country_name'].astype(str)

# Verify the data type changes
print(df[['employee_country_name', 'company_country_name']].dtypes)

##  Check Country Name Matching Between DataFrame and GeoJSON

In [ ]:
import json

# Load your GeoJSON file
file_path = '/Users/buketoztekin/Documents/world_countries.json'

with open(file_path, 'r') as f:
    try:
        geo_json_data = json.load(f)
    except json.JSONDecodeError as e:
        print(f"Error loading JSON file: {e}")
        geo_json_data = None

# Check if the GeoJSON was loaded successfully
if geo_json_data and 'features' in geo_json_data:
    # Extract all country names from the GeoJSON file
    geojson_countries = []
    
    # Loop through features to extract the country names
    for feature in geo_json_data['features']:
        if 'properties' in feature and 'name' in feature['properties']:
            geojson_countries.append(feature['properties']['name'])
        else:
            print(f"Missing 'properties' or 'name' in feature: {feature}")
    
    # Print the list of country names
    print(geojson_countries)
else:
    print("The 'features' key does not exist in the GeoJSON file or the file failed to load.")

In [ ]:
# Check unique country names in 'company_country_name' column
company_countries = df['company_country_name'].unique()
print("Unique country names in 'company_country_name':")
print(company_countries)

# Check unique country names in 'employee_country_name' column
employee_countries = df['employee_country_name'].unique()
print("\nUnique country names in 'employee_country_name':")
print(employee_countries)

### There are a few mismatch. To correct this, I will create a dictionary that maps the country names in the DataFrame to those in the GeoJSON file.

In [ ]:
# Create a dictionary to map country names from your dataset to those in the GeoJSON
country_name_mapping = {
    'United States': 'United States of America',
    'Türkiye': 'Turkey',
    'Viet Nam': 'Vietnam',
    'Korea, Republic of': 'South Korea',
    'Russian Federation': 'Russia',
    'Czechia': 'Czech Republic',
    'Iran, Islamic Republic of': 'Iran',
    'Moldova, Republic of': 'Moldova',
    'Bolivia, Plurinational State of': 'Bolivia',
    'Iran, Islamic Republic of': 'Iran'
}

# Apply the mapping to fix country names in your DataFrame
df['company_country_name'] = df['company_country_name'].replace(country_name_mapping)
df['employee_country_name'] = df['employee_country_name'].replace(country_name_mapping)

In [ ]:
# Check unique country names in 'company_country_name' column
company_countries = df['company_country_name'].unique()
print("Unique country names in 'company_country_name':")
print(company_countries)

# Check unique country names in 'employee_country_name' column
employee_countries = df['employee_country_name'].unique()
print("\nUnique country names in 'employee_country_name':")
print(employee_countries)

## Creating a Choropleth Map

I will create the map based on the average salary per country, using the company's location as the reference point.

In [ ]:
# Create a data frame with just the company countries and the salaries

data_to_plot = df[['company_country_name','salary_in_usd']]
data_to_plot.head()

### Choropleth map for average salary and company location

In [ ]:
# Calculate average salary per country
average_salary_per_country = data_to_plot.groupby('company_country_name')['salary_in_usd'].mean().reset_index()

# Rename columns for clarity
average_salary_per_country.columns = ['company_country_name', 'average_salary_in_usd']

# Check the result
print(average_salary_per_country.head())

In [ ]:
#Check for any missing values in the average salary data
missing_data = average_salary_per_country[average_salary_per_country['average_salary_in_usd'].isna()]
print("Countries with missing salary data:", missing_data)

In [ ]:
# Setup the folium map, center the map on the world
map = folium.Map(location=[20, 0], zoom_start=2)

# Create the Choropleth map with the average salary
choropleth = folium.Choropleth(
    geo_data=geo_json_data,  # Loaded GeoJSON data
    data=average_salary_per_country,  # Data with the average salary
    columns=['company_country_name', 'average_salary_in_usd'],  # Use average salary column
    key_on='feature.properties.name',  # Matching country names from GeoJSON
    fill_color='RdPu',  # Color scale
    fill_opacity=0.6,  # Opacity of the fill
    line_opacity=0.2,  # Opacity of the country borders
    legend_name="Average Salary in USD",  # Legend title
    nan_fill_color="white",  # Color for missing values
    highlight=True  # Highlight feature when hovering
).add_to(map)

# Add a layer control for toggling map elements (optional)
folium.LayerControl().add_to(map)

# Save the map to an HTML file
map.save("choropleth_map_average_salary.html")

# Display the map
map

The highest average salaries are paid in Qatar, followed by Israel, Saudi Arabia, the United States, Canada, Australia, Egypt, and Switzerland. In Europe, eastern and southern countries tend to offer lower salaries compared to the rest of the continent. Meanwhile, Mexico and Colombia stand out in Central and South America, offering higher average salaries than their regional counterparts.

### Choropleth map for maximum salary and company location

In [ ]:
# Calculate maximum salary per country
maximum_salary_per_country = data_to_plot.groupby('company_country_name')['salary_in_usd'].max().reset_index()

# Rename columns for clarity
maximum_salary_per_country.columns = ['company_country_name', 'maximum_salary_in_usd']

# Check the result
print(maximum_salary_per_country.head())

In [ ]:
# Setup the folium map, center the map on the world
map = folium.Map(location=[20, 0], zoom_start=2)

# Create the Choropleth map with the average salary
choropleth = folium.Choropleth(
    geo_data=geo_json_data,  # Loaded GeoJSON data
    data=maximum_salary_per_country,  # Data with the average salary
    columns=['company_country_name', 'maximum_salary_in_usd'],  # Use maximum salary column
    key_on='feature.properties.name',  # Matching country names from GeoJSON
    fill_color='RdPu',  # Color scale
    fill_opacity=0.6,  # Opacity of the fill
    line_opacity=0.2,  # Opacity of the country borders
    legend_name="Maximum Salary in USD",  # Legend title
    nan_fill_color="white",  # Color for missing values
    highlight=True  # Highlight feature when hovering
).add_to(map)

# Add a layer control for toggling map elements (optional)
folium.LayerControl().add_to(map)

# Save the map to an HTML file
map.save("choropleth_map_maximum_salary.html")

# Display the map
map

The highest-paid individuals are in the USA and Qatar, followed by the UK. This suggests a significant gap between the average and maximum salaries, or greater salary variation within these countries. A similar pattern appears in Russia, which shows a darker shade on the "maximum salary" map but a lighter shade on the "average salary" map compared to other countries.

In [ ]:
# Calculate the average remote work ratio by country (using company location)
average_remote_work_by_country = df.groupby('company_country_name')['remote_ratio'].mean().reset_index()

# Rename the columns for clarity
average_remote_work_by_country.columns = ['company_country_name', 'average_remote_ratio']

# Check the result
print(average_remote_work_by_country.head())

In [ ]:
average_remote_work_by_country

In [ ]:
# Setup the folium map, center the map on the world
remote_work_map = folium.Map(location=[20, 0], zoom_start=2)

# Add choropleth layer for average remote work ratio
choropleth = folium.Choropleth(
    geo_data=geo_json_data,  # Loaded GeoJSON data
    data=average_remote_work_by_country,  # Data with the average remote work ratio
    columns=['company_country_name', 'average_remote_ratio'],  # Use average remote work ratio
    key_on='feature.properties.name',  # Matching country names from GeoJSON
    fill_color='PuRd',  # Use a Purple-Red color scale
    fill_opacity=0.6,  # Opacity of the fill
    line_opacity=0.2,  # Opacity of the country borders
    legend_name="Average Remote Work Ratio",
    nan_fill_color="lightgray",  # Color for missing values
    highlight=True  # Highlight feature when hovering
).add_to(remote_work_map)

# Add a layer control for toggling map elements (optional)
folium.LayerControl().add_to(remote_work_map)

# Save the map to an HTML file
remote_work_map.save("average_remote_work_map.html")

# Display the map
remote_work_map

Countries with a higher average remote work ratio include Brazil, Argentina, Chile, Mexico, Greece, Ecuador, Sweden, and Poland, where companies tend to have more remote workers compared to others.

In [ ]:
df

## Additional analysis without the extreme salary values for 2023 - 2024

In my earlier analysis, I created the choropleth maps using salary data from all available years (2020, 2021, 2022, 2023, and 2024). However, the majority of the data points were concentrated in the recent years—especially in 2024, followed by 2023 and 2022. There was very little data for 2020 and 2021, which could skew the results when comparing trends across all years.

Given this, I decided to focus on 2023 and 2024 to capture the most recent trends in AI/ML job salaries. This approach allows me to provide a clearer and more accurate picture of the current salary landscape without the influence of outdated data.

#### Handling Outliers in Salary Data
Additionally, upon reviewing the data, I identified that some salaries were extremely high, likely representing outliers. These outliers can distort the overall analysis by inflating averages and giving a skewed perception of typical salaries in this field. To address this issue, I applied a quartile-based filtering approach, where I removed extreme values by focusing on the interquartile range (IQR). This method helps to keep the data within a more reasonable range, making the map more reflective of the general salary distribution without the influence of a few high salaries.

#### Objective of the New Maps
This new map, focusing on 2023–2024 data, provides a more accurate, up-to-date view of AI/ML job salaries across countries, adjusted to exclude extreme outliers. This way, it highlights more representative salary data and recent market trends, offering a clearer understanding of the current job market without distortion from either outdated data or unusually high salary points.

In [ ]:
df['salary_in_usd'].describe()

In [ ]:
# Given values
mean_salary = 146558.036405
std_salary = 69283.705335

# Define the threshold for extreme values (mean ± 2 * standard deviation)
upper_threshold = mean_salary + 2 * std_salary
lower_threshold = mean_salary - 2 * std_salary

print("Upper Threshold (mean + 2*std):", upper_threshold)
print("Lower Threshold (mean - 2*std):", lower_threshold)


In [8]:
# Remove the extreme salary values based on the thresholds
df_filtered = df[(df['salary_in_usd'] <= 285125.44707500003) & 
                                (df['salary_in_usd'] >= 7990.62573499998)]

# Check the size of the cleaned dataset
print("Original Size:", len(df))
print("Cleaned Size:", len(df_filtered))

KeyError: 'salary_in_usd'

In [ ]:
sns.distplot(df_filtered['salary_in_usd'], bins=25)

In [ ]:
df_filtered_2023_2024 = df_filtered[(df_filtered['work_year'] >= 2023) & (df_filtered['work_year'] <= 2024)]
# Print the unique values from the 'work_year' column
print("Unique values in 'work_year':", df_filtered_2023_2024['work_year'].unique())

## Creating New Choropleth Maps Based on Filtered Salary Data

In [ ]:
# Create a data frame with just the company countries and the salaries

data_to_plot2 = df_filtered_2023_2024[['company_country_name','salary_in_usd']]
data_to_plot2.head()

In [ ]:
# Group by company_country_name and calculate the average salary
average_salary_per_country2 = data_to_plot2.groupby('company_country_name').mean().reset_index()
average_salary_per_country2.columns = ['company_country_name', 'average_salary_in_usd']  

# Check the result
average_salary_per_country2.head()

In [ ]:
# Setup the folium map, center the map on the world
map = folium.Map(location=[20, 0], zoom_start=2)

# Create the Choropleth map with the average salary
choropleth = folium.Choropleth(
    geo_data=geo_json_data,  # Loaded GeoJSON data
    data=average_salary_per_country2,  # Data with the average salary
    columns=['company_country_name', 'average_salary_in_usd'],  # Use average salary column
    key_on='feature.properties.name',  # Matching country names from GeoJSON
    fill_color='OrRd',  # Color scale
    fill_opacity=0.6,  # Opacity of the fill
    line_opacity=0.2,  # Opacity of the country borders
    legend_name="Average Salary in 2023 & 2024 (in USD)",  # Legend title
    nan_fill_color="white",  # Color for missing values
    highlight=True  # Highlight feature when hovering
).add_to(map)

# Add a layer control for toggling map elements (optional)
folium.LayerControl().add_to(map)

# Save the map to an HTML file
map.save("choropleth_map_average_salary_2023&2024.html")

# Display the map
map

In [ ]:
# Group by company_country_name and calculate the average salary
maximum_salary_per_country2 = data_to_plot2.groupby('company_country_name').max().reset_index()
maximum_salary_per_country2.columns = ['company_country_name', 'maximum_salary_in_usd']  

# Check the result
maximum_salary_per_country2.head()

In [ ]:
# Setup the folium map, center the map on the world
map = folium.Map(location=[20, 0], zoom_start=2)

# Create the Choropleth map with the average salary
choropleth = folium.Choropleth(
    geo_data=geo_json_data,  # Loaded GeoJSON data
    data=maximum_salary_per_country2,  # Data with the average salary
    columns=['company_country_name', 'maximum_salary_in_usd'],  # Use average salary column
    key_on='feature.properties.name',  # Matching country names from GeoJSON
    fill_color='OrRd',  # Color scale
    fill_opacity=0.6,  # Opacity of the fill
    line_opacity=0.2,  # Opacity of the country borders
    legend_name="Maximum Salary in 2023 & 2024 (in USD)",  # Legend title
    nan_fill_color="white",  # Color for missing values
    highlight=True  # Highlight feature when hovering
).add_to(map)

# Add a layer control for toggling map elements (optional)
folium.LayerControl().add_to(map)

# Save the map to an HTML file
map.save("choropleth_map_average_salary_2023&2024.html")

# Add a layer control for toggling map elements (optional)
folium.LayerControl().add_to(map)

# Save the map to an HTML file
map.save("choropleth_map_average_salary_2023_2024.html")

# Display the map
map

# Exporting data

In [ ]:
#Export the new dataframe with the country columns
df.to_csv('/Users/buketoztekin/Documents/Cleaned_salaries_with_country_columns.csv', index=False)